<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

## Complément - niveau intermédiaire

Souvenez-vous de ce qu'on avait dit en semaine 3 séquence 4, concernant les clés dans un dictionnaire ou les éléments dans un ensemble. Nous avions vu alors que, pour les types *builtin*, les clés devaient être des objets immuables et même globalement immuables.

Nous allons voir dans ce complément quelles sont les règles qui s'appliquent aux instances de classe, et notamment  comment on peut manipuler des ensembles d'instances d'une manière qui fasse du sens.

Une instance de classe est presque toujours un objet mutable (voir à ce sujet un prochain complément sur les `namedtuple`s).

Et pourtant, le langage vous permet d'insérer une instance dans un ensemble - ou de l'utiliser comme clé dans un dictionnaire.

Nous allons voir ce mécanisme en action, et mettre en évidence ses limites.

### hachage par défaut: basé sur `id()`

In [ ]:
# une classe Point qui ne redéfinit pas __eq__ ni __hash__
class Point1:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Pt[{self.x}, {self.y}]"

In [ ]:
# deux instances 
p1 = Point1(2, 3)
p2 = Point1(3, 4)

# bien qu'ils soient mutables, on peut les mettre dans un ensemble
s = {p1, p2}

Mais par contre soyez attentifs, car il faut savoir que pour la classe `Point1`, où nous n'avons rien redéfini, la fonction de hachage sur une instance de `Point1` ne dépend que de la valeur de `id()` sur cet objet.

Ce qui, dit autrement, signifie que deux objets qui sont distincts au sens de `id()` sont considérés comme différents, et donc peuvent coexister dans un ensemble, ou dans un dictionnaire, ce qui n'est pas forcément ce qu'on veut:

In [ ]:
# un point similaire à p1
p0 = Point1(2, 3)
# nos deux objets se ressemblent
p0, p1

In [ ]:
# mais peuvent coexister dans un ensemble
{ p0, p1 }

### `__hash__` et `__eq__` 

Le protocole hashable permet de pallier à cette déficience; pour cela il nous faut définir deux méthodes:

* `__eq__` qui, sans grande surprise, va servir à évaluer `p == q` 
* et `__hash__` qui va retourner la clé de hachage sur un objet

La subtilité étant bien entendu que ces deux méthodes doivent être cohérentes; c'est-à-dire que, de bon sens, si l'égalité se base sur nos deux attributs `x` et `y`, il faudra bien entendu que la fonction de hachage utilise elle aussi ces deux attributs, pour que la logique de la table de hachage (semaine 3 séquence 3) fonctionne de manière satisfaisante.

Voyons cela sur une sous-classe de `Point1`, dans laquelle nous définissons ces deux méthodes:

In [ ]:
class Point2(Point1):

    # l'égalité va se baser naturellement sur x et y
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    # du coup la fonction de hachage aussi
    def __hash__(self):
        return (11 * self.x + self.y) // 16

On peut vérifier que cette fois les choses fonctionnent correctement:

In [ ]:
q0 = Point2(2, 3)
q1 = Point2(2, 3)

In [ ]:
# nos deux objets sont distincts pour is() mais égaux pour ==
print(f"is → {q0 is q1} \n== → {q0 == q1}")

In [ ]:
# un ensemble contenant les deux points n'en contient qu'un
{q0, q1}

In [ ]:
# et bien sûr c'est pareil pour un dictionnaire
d = {}
d[q0] = 1
# les deux clés q0 et q1 sont les mêmes pour le dictionnaire
# du coupe ici on écrase la (seule) valeur dans d
d[q1] = 10000
d